In [134]:
import random

#random.seed(42)

def create_graph(cols=10, max_height=20):
    """Create a random bar chart of width 1, and length cols."""
    height = [random.randint(0,max_height) for i in range(cols)]
    width = [1 for i in range(cols)]
    area = [height[i] * width[i] for i in range(cols)]
    
    return list(zip(height, width, area))

def compress(graph, start_pos, count):
    """For the subset of graphs, find the min height and multuple by the sum of the widths"""
    graph_subset = graph[start_pos:start_pos+count+1]
    #print("graph_subset: " +  str(graph_subset))
    min = None
    width = 0
    for tuple in graph_subset:
        h, w, a = tuple[0:3]
        width += w
        if min is None:
            min = h
        elif h < min:
            min = h
    return min * width

def find_max(graph):
    """
    Brute force algorithm to find the biggest rectangle in a graph.
    From every position check the next positions, and calculate
    the maximum area using compress(). Stop when this starts to reduce,
    and move onto the next starting position. Also stop when any areas
    are zero.
    
    It's time O(N^3) - or at best O(N^2.logN), space O(N).
    """
    loop_max = []
    size = len(graph)
    for start in range(size):
        largest = 0
        end = size - start
        #print("start: " + str(start))
        #print("end: " + str(end))
        for count in range(end):
            area = compress(graph, start, count)
            #print("area: " + str(area))
            if area == 0:
                break
            elif area > largest:
                largest = area
        loop_max.append(largest)
    return max(loop_max)

graph = create_graph(cols=5, max_height=10)
print(find_max(graph))
#print(graph)

30


Let's create some unit tests:

In [136]:
import unittest

class TestGraph(unittest.TestCase):
        
    def test_all_zero(self):
        height = [0] * 10
        width  = [1] * 10
        area   = [0] * 10
        graph  = list(zip(height, width, area))
        self.assertEqual(find_max(graph), 0)

    def test_all_1(self):
        height = [1] * 10
        width  = [1] * 10
        area   = [1] * 10
        graph  = list(zip(height, width, area))
        self.assertEqual(find_max(graph), 10)
        
    def test_alternating_10(self):
        height = [10, 0] * 5
        width  = [1] * 10
        area   = [height[i] * width[i] for i in range(10)]
        graph  = list(zip(height, width, area))
        self.assertEqual(find_max(graph), 10)
        
    def test_staircase_up_10(self):
        height = list(range(11))
        width  = [1] * 10
        area   = [height[i] * width[i] for i in range(10)]
        graph  = list(zip(height, width, area))
        self.assertEqual(find_max(graph), 25)

    def test_staircase_down_10(self):
        height = list(range(9, -1, -1))
        width  = [1] * 10
        area   = [height[i] * width[i] for i in range(10)]
        graph  = list(zip(height, width, area))
        self.assertEqual(find_max(graph), 25)
        
    def test_teeth_10(self):
        height = [0,10,10,0,21,0,5,6,6,5]
        width  = [1] * 10
        area   = [height[i] * width[i] for i in range(10)]
        graph  = list(zip(height, width, area))
        self.assertEqual(find_max(graph), 21)

    def test_pyramid_10(self):
        height = [0,1,2,3,4,4,3,2,1,0]
        width  = [1] * 10
        area   = [height[i] * width[i] for i in range(10)]
        graph  = list(zip(height, width, area))
        self.assertEqual(find_max(graph), 12)

    def test_flipped_pyramid_10(self):
        height = [4,3,2,1,0,0,1,2,3,4]
        width  = [1] * 10
        area   = [height[i] * width[i] for i in range(10)]
        graph  = list(zip(height, width, area))
        self.assertEqual(find_max(graph), 6)
        
    def test_tall_and_flat_10(self):
        height = [10,0,0,0,0,2,2,2,2,2]
        width  = [1] * 10
        area   = [height[i] * width[i] for i in range(10)]
        graph  = list(zip(height, width, area))
        self.assertEqual(find_max(graph), 10)
        
    def test_sawtooth_10(self):
        height = [3,9,18,9,3,9,18,9,3,9]
        width  = [1] * 10
        area   = [height[i] * width[i] for i in range(10)]
        graph  = list(zip(height, width, area))
        self.assertEqual(find_max(graph), 30)
    

a = TestGraph()
suite = unittest.TestLoader().loadTestsFromModule(a)
unittest.TextTestRunner().run(suite)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.023s

OK


<unittest.runner.TextTestResult run=10 errors=0 failures=0>

A more efficient algorithm - we can take advantage of commutation. I can replace [3,6,3] with [3,3,3]. or 3 with a width 3.

Find chunks we can compress. Then replace those entries with a single entry. e.g.
```
[2, 2, 0, 4, 5]
[1, 1, 1, 1, 1]
[2, 2, 0, 4, 5]
```

Compress the first two:
```
[2, 0, 4, 5]
[2, 1, 1, 1]
[4, 1, 4, 5]
```

Compress the last two:
```
[2, 0, 4]
[2, 1, 2]
[4, 0, 8]
```

No more compressions possible, pick the largest value:
```
[4]
[2]
[8]
```